# Experiment

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

text = open("data/input.txt", "r")

# get unique chars in training data
chars = sorted(list(set(text)))
vocab_size = len(chars)

# map string to integers (idx)
stoi = { ch:i for i,ch in enumerate(chars) } # string to int
itos = { i:ch for i,ch in enumerate(chars) } # int to string

# char encode/decode
encode = lambda s: [stoi[c] for c in s] # string -> list of ints
decode = lambda l: ''.join([itos[i] for i in l]) # list of ints -> string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in ix])
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# hyperparameters
batch_size = 64
context_length = 256
max_iters = 5000
eval_interval = 250
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384 # for token embedding dimension
n_head = 6 # n_embd/n_head = 384/6 = every head is 64 dimensional (token embedding is divided evenly among the attention heads)
n_layer = 6
dropout = 0.2


In [5]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(context_length, context_length)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out # (B, T, C)

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # multiple heads in parallel
        self.proj = nn.Linear(n_embd, n_embd) # residual connection
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # output of self-attention
        out = self.dropout(self.proj(out)) # projection back into residual pathway
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # inner-layer has multiplier of 4 for more neurons that does computing (just performed best empirically according to "attention is all you need" paper)
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # projection layer going back into residual pathway
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class TransformerBlock(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # data communication
        x = x + self.ffwd(self.ln2(x)) # data computation
        return x

class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(context_length, n_embd)
        self.blocks = nn.Sequential(*[TransformerBlock(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B,T,n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,n_embd)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        if targets is None:
            loss = None # inference mode
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -context_length:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = LanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


10.788929 M parameters
step 0: train loss 4.2849, val loss 4.2819
step 250: train loss 2.3827, val loss 2.4084
step 500: train loss 2.0118, val loss 2.1025
step 750: train loss 1.7486, val loss 1.8884
step 1000: train loss 1.5967, val loss 1.7738
step 1250: train loss 1.5063, val loss 1.6962
step 1500: train loss 1.4340, val loss 1.6401
step 1750: train loss 1.3815, val loss 1.6032
step 2000: train loss 1.3405, val loss 1.5619
step 2250: train loss 1.3071, val loss 1.5487
step 2500: train loss 1.2760, val loss 1.5277
step 2750: train loss 1.2468, val loss 1.5179
step 3000: train loss 1.2248, val loss 1.5037
step 3250: train loss 1.2054, val loss 1.4958
step 3500: train loss 1.1857, val loss 1.4901
step 3750: train loss 1.1636, val loss 1.4852
step 4000: train loss 1.1464, val loss 1.4858
step 4250: train loss 1.1277, val loss 1.4808
step 4500: train loss 1.1100, val loss 1.4791
step 4750: train loss 1.0905, val loss 1.4853
step 4999: train loss 1.0741, val loss 1.4840


In [6]:
# generate text
starting_prompt = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(starting_prompt, max_new_tokens=2000)[0].tolist()))


BUCKINGHAM:
God call these satisfate, forewell.

GEORGE:
My lord, God, forbid an else purse vows;
For I impt you true in your posts,
Being the fear by yonderable, these weep,
Or most lightly work I heart of eyes,
And what would not see 'em the gods on my sent?

BISHOP OF ELY:
And to-day I do but onfer Montague;
There did princely armour tied of A moriof;
Who bidun the prince looks on 's earth. Nay, Camillo well'd
That mayst thwice to him made use the Eas.

BETH:
Overefore, farewell.

HENRY BOLINGBROKE:
Accout of for pride your grace my faitht.

BUSHY:
Bless a diremned to peece for disgrace I straint.

DORCASE:
Wear men, after for I do live him, save a
voice by minister, one is land,
And this dangerous give sorrow was with already,
And swell his great accession can beggard,
Lest his pardon words. Now, very heart you:
Prone, sir, heaven she, a widoward it his honest.

CAPULET:
Ha! stable power mournists her poper.

PAGE:
Go of me your look; for he's see so, my lord.

ANGELO:
Well; I ne'